In [ ]:
effdet_no = 5
try :
    effdet_no = int(input("What efficientdet do you want to use for prediction? (0/1/2/3/4/5/6/7) \n"))
except :
    effdet_no = 5
if effdet_no>7 :
    effdet_no = 5
print("Using Effdet",effdet_no)


What efficientdet do you want to use for prediction? (0/1/2/3/4/5/6/7) 
5
Using Effdet 5


In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
!pip install effdet==0.1.6

     |████████████████████████████████| 61kB 5.3MB/s 
     |████████████████████████████████| 245kB 10.7MB/s 


In [ ]:
# !pip install torch==1.5.0+cu101 torchvision==0.6.0+cu101 -f https://download.pytorch.org/whl/torch_stable.html

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# ! git clone https://github.com/NVIDIA/apex
# os.chdir('apex')
# ! pip install -v --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ./
# os.chdir('../')

In [ ]:
import torch

In [ ]:
#!pip install torch==1.5.0+cu101 torchvision==0.6.0+cu101 -f https://download.pytorch.org/whl/torch_stable.html
!pip install albumentations==0.4.6

     |████████████████████████████████| 122kB 11.2MB/s 
     |████████████████████████████████| 952kB 18.6MB/s 
  Created wheel for albumentations: filename=albumentations-0.4.6-cp36-none-any.whl size=65165 sha256=3226611d3a1d082394054d3ae868934ffc6c5fdef747b64f5c471825315786db
  Stored in directory: /root/.cache/pip/wheels/c7/f4/89/56d1bee5c421c36c1a951eeb4adcc32fbb82f5344c086efa14
Successfully built albumentations
  Found existing installation: imgaug 0.2.9
    Uninstalling imgaug-0.2.9:
      Successfully uninstalled imgaug-0.2.9
  Found existing installation: albumentations 0.1.12
    Uninstalling albumentations-0.1.12:
      Successfully uninstalled albumentations-0.1.12


In [ ]:
import albumentations as A
# from apex import amp
from albumentations.pytorch.transforms import ToTensorV2
import albumentations as A
from albumentations.augmentations.transforms import RandomRain, RandomSnow, RandomFog
from albumentations.pytorch.transforms import ToTensorV2

In [ ]:
import sys
sys.path.insert(0, "/content/timm_efficientdet_pytorch")
sys.path.insert(0, "/content/omegaconf")
sys.path.insert(0, "/content/weightedboxesfusion")

import os
from datetime import datetime
import time
import random
import cv2
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedKFold
from torch.utils.data import Dataset,DataLoader
from torch.utils.data.sampler import SequentialSampler, RandomSampler

from glob import glob
from skimage import io


np.random.seed(5)

In [ ]:
TEST_ROOT_PATH = '/content/drive/My Drive/Dhaka-AI 2020/dataset/test round 2'

In [ ]:

!nvidia-smi

Mon Dec  7 18:09:42 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.45.01    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install omegaconf

     |████████████████████████████████| 276kB 14.2MB/s 
  Created wheel for PyYAML: filename=PyYAML-5.3.1-cp36-cp36m-linux_x86_64.whl size=44619 sha256=2133add48ce317e744c9134554669b0c1b28aa67de2362210ecaa727c76059de
  Stored in directory: /root/.cache/pip/wheels/a7/c1/ea/cf5bd31012e735dc1dfea3131a2d5eae7978b251083d6247bd
Successfully built PyYAML
  Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
!pip install ensemble-boxes

In [ ]:
sys.path.insert(0, "/content/weightedboxesfusion")

from ensemble_boxes import *
from effdet import get_efficientdet_config, EfficientDet, DetBenchPredict
from effdet.efficientdet import HeadNet
import gc
import matplotlib.pyplot as plt




RuntimeError: ignored

In [ ]:
from random import sample
IMG_SIZE = 512
DATA_ROOT_PATH = "/content/drive/My Drive/Dhaka-AI 2020/dataset/test round 2/"
test_images = sorted(glob(DATA_ROOT_PATH + '*'))
from random import sample

def get_ids(path_list):
  path_list = random.sample(path_list,len(path_list))
  id_list = [path.split('/')[-1][:-4] for path in path_list]
  return np.array(id_list)

valid_ids = get_ids(test_images)

print(len(valid_ids))
# print(valid_ids)

In [ ]:
def get_test_transforms():
    return A.Compose([
            A.Resize(height=IMG_SIZE, width=IMG_SIZE, p=1.0),
            ToTensorV2(p=1.0),
        ], p=1.0)

In [ ]:
class DatasetRetriever(Dataset):

    def __init__(self, image_ids, transforms=None):
        super().__init__()
        self.image_ids = image_ids
        self.transforms = transforms

    def __getitem__(self, index: int):
        image_id = self.image_ids[index]
        image = cv2.imread(f'{TEST_ROOT_PATH}/{image_id}.jpg', cv2.IMREAD_COLOR)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB).astype(np.float32)
        image /= 255.0
        if self.transforms:
            sample = {'image': image}
            sample = self.transforms(**sample)
            image = sample['image']
        return image, image_id

    def __len__(self) -> int:
        return self.image_ids.shape[0]

In [ ]:
test_data = DatasetRetriever(
    image_ids=valid_ids,
    transforms=get_test_transforms()
)

def collate_fn(batch):
    return tuple(zip(*batch))

data_loader = DataLoader(
    test_data,
    batch_size=1,
    shuffle=False,
    num_workers=4,
    drop_last=False,
    collate_fn=collate_fn)

In [ ]:
CHECKPOINT_PATH = '/content/drive/MyDrive/Dhaka-AI 2020/effdet_best_weights/effdet5_512_4'

In [ ]:
get_efficientdet_config??

Object `get_efficientdet_config` not found.


In [ ]:
Epoch = 0

def load_net(checkpoint_path):
    config = get_efficientdet_config('tf_efficientdet_d'+str(effdet_no))
    net = EfficientDet(config, pretrained_backbone=False)
    config.num_classes = 21
    config.image_size=IMG_SIZE
    net.class_net = HeadNet(config, num_outputs=config.num_classes, norm_kwargs=dict(eps=.001, momentum=.01))

    checkpoint = torch.load(checkpoint_path)
    net.load_state_dict(checkpoint['model_state_dict'])
    Epoch = checkpoint['epoch']
    print(Epoch)

    del checkpoint
    gc.collect()

    net = DetBenchPredict(net, config)
    net.eval();
    return Epoch,net.cuda()

Epoch,net = load_net(CHECKPOINT_PATH +'/last-checkpoint.bin')

In [ ]:
def make_predictions(images, score_threshold=0.23):
    
    images = torch.stack(images).cuda().float()
    predictions = []
    img_scale = torch.tensor([[1.] for _ in range(images.shape[0])]).cuda().float()
    img_size = torch.tensor([(IMG_SIZE, IMG_SIZE) for _ in range(images.shape[0])]).cuda().float()
        
    with torch.no_grad():
        det = net(images,img_scales = img_scale,img_size = img_size)
        for i in range(images.shape[0]):
            boxes = det[i].detach().cpu().numpy()[:,:4]   
            ##----------CHECK HERE 1------------- 
            scores = det[i].detach().cpu().numpy()[:,4]
            ##
            classes = det[i].detach().cpu().numpy()[:,5]

            ##----------CHECK HERE 2------------- 
            indexes = np.where(scores > score_threshold)[0]
            ##
            boxes = boxes[indexes]
            boxes[:, 2] = boxes[:, 2] + boxes[:, 0]
            boxes[:, 3] = boxes[:, 3] + boxes[:, 1]
            predictions.append({
                'boxes': boxes[indexes],
                'scores': scores[indexes],
                'classes': classes[indexes]
            })
    return [predictions]

def run_wbf(predictions, image_index, image_size=IMG_SIZE, iou_thr=0.44, skip_box_thr=0.43, weights=None):
    boxes = [(prediction[image_index]['boxes']/(image_size-1)).tolist()  for prediction in predictions]
    scores = [prediction[image_index]['scores'].tolist()  for prediction in predictions]
    labels = [prediction[image_index]['classes'].tolist()  for prediction in predictions]
    ##----------CHECK HERE 3------------- 
    boxes, scores, labels = weighted_boxes_fusion(boxes, scores, labels, weights=None, iou_thr=iou_thr, skip_box_thr=skip_box_thr)
    boxes = boxes*(image_size-1)
    return boxes, scores, labels

In [ ]:

# for k in range(0,10):
#   i = np.random.RandomState().randint(100)

#   for j, (images, image_ids) in enumerate(data_loader):
#       if j == i:
#           break
#   print(image_ids)
#   predictions = make_predictions(images)
#   i = 0
#   sample = images[0].permute(1,2,0).cpu().numpy()

#   boxes, scores, labels = run_wbf(predictions, image_index=i)
#   boxes = boxes.astype(np.int32).clip(min=0, max=639)

#   fig, ax = plt.subplots(1, 1, figsize=(16, 8))

#   for box in boxes:
#       cv2.rectangle(sample, (box[0], box[1]), (box[2], box[3]), (1, 0, 0), 1)
    
#   ax.set_axis_off()
#   ax.imshow(sample);
    
    

In [ ]:
# def showpred(id):
    
#     dataset = DatasetRetriever(
#     image_ids=np.array([id]),
#     transforms=get_valid_transforms()
#        )



#     data_loader = DataLoader(
#     dataset,
#     batch_size=1,
#     shuffle=False,
#     num_workers=4,
#     drop_last=False,
#     collate_fn=collate_fn)

#     import matplotlib.pyplot as plt

#     for j, (images, image_ids) in enumerate(data_loader):
#         break

#     predictions = make_predictions(images)

#     i = 0
#     sample = images[i].permute(1,2,0).cpu().numpy()

#     boxes=predictions[0][0]['boxes']
#     scores=predictions[0][0]['scores']
#     labels=predictions[0][0]['classes']
#     boxes = boxes.astype(np.int32).clip(min=0, max=511)

#     fig, ax = plt.subplots(1, 1, figsize=(16, 8))

#     for box in boxes:
#         cv2.rectangle(sample, (box[0], box[1]), (box[2], box[3]), (1, 0, 0), 1)
    
#     ax.set_axis_off()
#     ax.imshow(sample);
    
        
    
# from PIL  import  Image
# def showgt(id):
    
#     img = cv2.imread(f'{TEST_ROOT_PATH}/{image_id}.jpg')
#       # opencv always read image as BGR - conversion is must
#     img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

#     arr=marking_test[marking_test['image_id']==id]
#     for x in range(len(arr)):
        
    
#         xm=arr['x'].iloc[x]
#         ym=arr['y'].iloc[x]
#         width=arr['w'].iloc[x]
#         height=arr['h'].iloc[x]
    
    
    
      
#         #xm,ym,width,height= box
#         xmin=xm
#         xmax=xm+width
#         ymin=ym
#         ymax=ym+height
#         # converting string to int

#         xmin= int(xmin); ymin= int(ymin)
#         xmax= int(xmax) ; ymax= int(ymax)

      

#         img= cv2.rectangle(img,(xmin,ymin), (xmax,ymax),(255,0,0),2)
#         img1=cv2.resize(img,(350,350))
      
#         img1 = Image.fromarray(img1) # cv2 to PIL
#     return img1

In [ ]:

from tqdm import tqdm
arr=[]
for j, (images, image_ids) in tqdm(enumerate(data_loader)):
    
    x=images[0]
    
    predictions = make_predictions(images)
    boxes=predictions[0][0]['boxes']
    scores=predictions[0][0]['scores']
    labels=predictions[0][0]['classes']
    '''for i, image in enumerate(images):
        boxes, scores, labels = run_wbf(predictions, image_index=i)
        boxes = (boxes).round().astype(np.int32).clip(min=0, max=511)
        image_id = image_ids[i]
        score_threshold = 0.09
        indexes = np.where(scores>score_threshold)
        boxes = boxes[indexes]
        scores = scores[indexes]
        labels = labels[indexes]'''
    
    
    #with open('/kaggle/working/valid_results/{}.txt'.format(image_ids[0]),'w') as f:
        
    
    for p  in range(len(boxes)):
        arr1=[]
        arr2=[]


        #f.write('car {} {} {} {} {}\n'.format(format(scores[p], ".5f"),int(boxes[p][0]),int(boxes[p][1]),int(boxes[p][2]),int(boxes[p][3])))
        arr1.append(image_ids[0])
        arr1.append(labels[p])
        arr1.append((format(scores[p], ".5f")))
        arr1.append(int(float(boxes[p][0])*(1024*1.0/IMG_SIZE)))
        arr1.append(int(float(boxes[p][1])*(1024*1.0/IMG_SIZE)))
        arr1.append(int(float(boxes[p][2])*(1024*1.0/IMG_SIZE)))
        arr1.append(int(float(boxes[p][3])*(1024*1.0/IMG_SIZE))) #[image.shape[1]]+ [image.shape[0]]
        arr1.append(int(1024))
        arr1.append(int(1024))
        arr.append(arr1)

In [ ]:
pd.DataFrame(data=arr,index=None,columns=['image_id','class','score','xmin','ymin','xmax','ymax','width','height'])

In [ ]:
df_pred=pd.DataFrame(data=arr,index=None,columns=['image_id','class','score','xmin','ymin','xmax','ymax','width','height'])
df_pred.to_csv(r'./pred_df.csv', index = False)

In [ ]:
len(df_pred['image_id'].unique())

In [ ]:
grouped_df = pd.read_csv('/content/drive/My Drive/Dhaka-AI 2020/dataset/classname_codes.csv')
grouped_df = grouped_df.drop(columns = ['Unnamed: 0'])
grouped_df

In [ ]:
left = df_pred
right = grouped_df

left['class'] = left['class'].map(right.set_index('class')['classname'])
test_result = left
test_result = test_result.drop_duplicates()

test_result

In [ ]:
test_result.to_csv('/content/drive/My Drive/Dhaka-AI 2020/Code/sadia_effdet_test_result/effdet'+str(effdet_no)+'512_4_'+str(Epoch)+'.csv',index=False)


In [ ]:
test_result[test_result['xmin']<0]

In [ ]:
test_result[test_result['xmax']>1024]

In [ ]:
test_result[test_result['ymin']<0]

In [ ]:
test_result[test_result['ymax']>1024]